In [3]:
import pandas as pd

In [ ]:
#Loading the uploaded files
file_2013 = "./data/ebird_data_wessan_2013_2014.csv"
file_2023 = "./data/ebird_data_wessan_2023_2024.csv"

df13 = pd.read_csv(file_2013)
df23 = pd.read_csv(file_2023)

#Checking for null values in data
print(f"Null values df13: \n{df13.isnull().sum()}")
print(f"\nNull values df23: \n{df23.isnull().sum()}")

Null values df13: 
subId                0
lat                  0
locationPrivate      0
locName              0
obsDt                0
sciName              0
howMany            145
locId                0
obsValid             0
speciesCode          0
obsReviewed          0
lng                  0
comName              0
dtype: int64

Null values df23: 
sciName             0
comName             0
locName             0
obsReviewed         0
obsValid            0
obsDt               0
lat                 0
howMany            78
subId               0
locationPrivate     0
lng                 0
speciesCode         0
locId               0
dtype: int64


In [15]:
df13.howMany = df13.howMany.fillna(1)
df23.howMany = df23.howMany.fillna(1)

In [60]:
total_birds_13 = df13.howMany.sum()
total_birds_23 = df23.howMany.sum()

print("Total birds observed 2013–14:", total_birds_13)
print("Total birds observed 2023–24:", total_birds_23)

Total birds observed 2013–14: 192869.0
Total birds observed 2023–24: 122038.0


In [78]:
# Seeing how many nodes we have have
# Converting the latitudes and longitudes into grids of size 2 degree
GRID_SIZE = 2.0

def assign_grid(df, size):
    df = df.copy()
    df["grid_lat"] = (df.lat / size).round().astype(int)
    df["grid_lng"] = (df.lng / size).round().astype(int)
    df["grid_cell"] = list(zip(df["grid_lat"], df["grid_lng"]))
    return df

In [79]:
df13g = assign_grid(df13, GRID_SIZE)
df23g = assign_grid(df23, GRID_SIZE)

print("\nGrid nodes:")
print("Number of nodes 2013–14:", df13g.grid_cell.nunique())
print("Number of nodes 2023–24:", df23g.grid_cell.nunique())


Grid nodes:
Number of nodes 2013–14: 82
Number of nodes 2023–24: 81


In [80]:
df23g.grid_cell

0       (17, -59)
1       (22, -62)
2       (24, -60)
3       (25, -62)
4       (21, -62)
          ...    
1700    (19, -61)
1701    (21, -61)
1702    (24, -60)
1703    (24, -62)
1704    (11, -53)
Name: grid_cell, Length: 1705, dtype: object

In [81]:
grid13 = set(df13g.grid_cell.unique())
grid23 = set(df23g.grid_cell.unique())

common_nodes = grid13.intersection(grid23)
# present in 2013 but missing in 2023
lost_nodes = grid13 - grid23
# appear only in 2023
new_nodes  = grid23 - grid13

print("Common nodes:", len(common_nodes))
print("Nodes only in 2013–14:", len(lost_nodes))
print("Nodes only in 2023–24:", len(new_nodes))

Common nodes: 64
Nodes only in 2013–14: 18
Nodes only in 2023–24: 17


In [82]:
print("Top 10 densest regions (2013–14):")
print(df13g.groupby("grid_cell").howMany.sum().sort_values(ascending=False).head(10))

print("\nTop 10 densest regions (2023–24):")
print(df23g.groupby("grid_cell").howMany.sum().sort_values(ascending=False).head(10))


Top 10 densest regions (2013–14):
grid_cell
(30, -73)    66816.0
(14, -56)    19651.0
(25, -61)    16957.0
(19, -61)    14987.0
(29, -68)    14030.0
(12, -54)    11926.0
(23, -62)     9786.0
(24, -61)     4409.0
(17, -59)     3785.0
(20, -62)     2923.0
Name: howMany, dtype: float64

Top 10 densest regions (2023–24):
grid_cell
(11, -53)    27322.0
(19, -61)    22879.0
(23, -62)    13051.0
(16, -57)     8888.0
(10, -53)     8684.0
(16, -58)     6918.0
(12, -53)     4462.0
(25, -61)     3483.0
(30, -73)     3007.0
(15, -58)     2344.0
Name: howMany, dtype: float64


In [ ]:
def normalize_datetime(df):
    #Converting to datetime where possible
    dt = pd.to_datetime(df["obsDt"], errors="coerce")

    # Identifying rows missing a timestamp by matching with regex to look for tt:tt
    has_time = df["obsDt"].str.contains(r"\d{2}:\d{2}", regex=True, na=False)

    #For rows without timestamp appending "00:00:00"
    dt_missing = pd.to_datetime(
        df.loc[~has_time, "obsDt"].astype(str).str.strip() + " 00:00:00",
        errors="coerce"
    )

    #Replace missing-time rows with new midnight timestamps
    dt.loc[~has_time] = dt_missing

    df["dateTime"] = dt
    return df

In [46]:
df13 = normalize_datetime(df13)
df23 = normalize_datetime(df23)


In [ ]:
# Chatgpt
df = df13  # or df23

# 1. Sort by time
df = df.sort_values("dateTime")

# 2. Bin by week
df["week"] = df["dateTime"].dt.isocalendar().week
df["year"] = df["dateTime"].dt.year

# Combine year+week to avoid mixing different years
df["year_week"] = df["year"].astype(str) + "-" + df["week"].astype(str)

# 3. Mean latitude and longitude per week
weekly = df.groupby("year_week")[["lat", "lng"]].mean().reset_index()

# 4. Compute directional movement vectors
weekly["lat_shift"] = weekly["lat"].diff()
weekly["lng_shift"] = weekly["lng"].diff()

print(weekly.head(15))


   year_week        lat         lng  lat_shift  lng_shift
0     2013-1  33.687831 -116.478126        NaN        NaN
1    2013-40  41.664776 -122.689149   7.976946  -6.211022
2    2013-41  40.156921 -120.118656  -1.507855   2.570492
3    2013-42  40.992566 -120.587039   0.835645  -0.468382
4    2013-43  35.357099 -116.971181  -5.635467   3.615857
5    2013-44  38.347954 -120.323892   2.990855  -3.352710
6    2013-45  40.132133 -120.348079   1.784178  -0.024187
7    2013-46  33.454607 -115.356503  -6.677525   4.991576
8    2013-47  37.317212 -118.563684   3.862604  -3.207181
9    2013-48  37.274821 -118.570899  -0.042391  -0.007216
10   2013-49  35.315106 -117.421270  -1.959715   1.149630
11   2013-50  33.514319 -116.400232  -1.800787   1.021037
12   2013-51  32.426426 -115.437079  -1.087893   0.963153
13   2013-52  37.083057 -118.865547   4.656631  -3.428468
14    2014-1  33.740910 -116.401947  -3.342147   2.463600
